In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.ops.focal_loss import sigmoid_focal_loss
from utils.model import ECG_XNOR_Full_Bin, ECG_XNOR_Full_Ori,  ECG_XNOR_Full_Ori_HW
from utils.OP import WeightOperation, FocalLoss, focal_loss
from utils.dataset import Loader
from utils.engine import train
from utils.save_model import save_model

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import numpy as np
import random
import os
# 110:92.97  170:92.97
classes_num = 5
test_size = 0.2
if classes_num == 17:
    batch_size = 64
    lr = 0.002
    seed = 142
else:
    batch_size = 512
    lr = 0.02
    seed = 101  #211


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

loader = Loader(batch_size=batch_size, classes_num=classes_num, device=device, test_size=test_size)
labels, train_loader, test_loader = loader.loader()
# in_channels, out_channels,    kernel_size,     stride,    padding,   pad_value,   pool_size,  pool_stride
kernel_size, padding, poolsize =7, 5, 7
padding_value = 1
A = [[1,           8,           kernel_size,       2,       padding,       padding_value,       poolsize,        2],
     [8,          16,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [16,         32,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [32,         32,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [32,         64,           kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     [64,         classes_num,  kernel_size,       1,       padding,       padding_value,       poolsize,        2],
     ]

model_ori = ECG_XNOR_Full_Ori(block1=A[0], block2=A[1], block3=A[2], block4=A[3],
                      block5=A[4] if len(A) > 4 else None,
                      block6=A[5] if len(A) > 5 else None,
                      block7=A[6] if len(A) > 6 else None,
                      device=device).to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
# loss_fn = FocalLoss(alpha=0.25, gamma=3)
optimizer = optim.Adam(model_ori.parameters(), lr=lr)
# loader.plot_train_test_splits()
print(device)
print(seed)

cuda:0
101


In [2]:
from torchinfo import summary
summary(model=model_ori,
        input_size=(batch_size, 1, 3600),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
ECG_XNOR_Full_Ori (ECG_XNOR_Full_Ori)         [64, 1, 3600]        [64, 17]             --                   True
├─Bn_bin_conv_pool_block_bw (block1)          [64, 1, 3600]        [64, 8, 898]         --                   True
│    └─ConstantPad1d (pad)                    [64, 1, 3600]        [64, 1, 3610]        --                   --
│    └─BinaryConv1d_bw (conv)                 [64, 1, 3610]        [64, 8, 1802]        56                   True
│    └─MaxPool1d (pool)                       [64, 8, 1802]        [64, 8, 898]         --                   --
│    └─PReLU (prelu)                          [64, 8, 898]         [64, 8, 898]         1                    True
│    └─BatchNorm1d (bn)                       [64, 8, 898]         [64, 8, 898]         16                   True
├─Bn_bin_conv_pool_block_baw (block2)         [64, 8, 898]         [64, 16, 448]       

In [3]:
from utils.Draw import plot_cfm
model_best = torch.load(f'./models/ECG_Net_for_{classes_num}_97.50%.pth')
summary(model=model_best,
        input_size=(batch_size, 1, 3600),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
ECG_XNOR_Full_Ori (ECG_XNOR_Full_Ori)         [64, 1, 3600]        [64, 17]             --                   True
├─Bn_bin_conv_pool_block_bw (block1)          [64, 1, 3600]        [64, 8, 898]         --                   True
│    └─ConstantPad1d (pad)                    [64, 1, 3600]        [64, 1, 3610]        --                   --
│    └─BinaryConv1d_bw (conv)                 [64, 1, 3610]        [64, 8, 1802]        56                   True
│    └─MaxPool1d (pool)                       [64, 8, 1802]        [64, 8, 898]         --                   --
│    └─PReLU (prelu)                          [64, 8, 898]         [64, 8, 898]         1                    True
│    └─BatchNorm1d (bn)                       [64, 8, 898]         [64, 8, 898]         16                   True
├─Bn_bin_conv_pool_block_baw (block2)         [64, 8, 898]         [64, 16, 448]       

In [4]:
from utils.model import  ECG_XNOR_Full_Ori_HW
model_HW = ECG_XNOR_Full_Ori_HW(block1=A[0], block2=A[1], block3=A[2], block4=A[3],
                      block5=A[4] if len(A) > 4 else None,
                      block6=A[5] if len(A) > 5 else None,
                      block7=A[6] if len(A) > 6 else None,
                      device=device).to(device)

In [5]:
summary(model=model_HW,
        input_size=(batch_size, 1, 3600),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
ECG_XNOR_Full_Ori_HW (ECG_XNOR_Full_Ori_HW)   [64, 1, 3600]        [64, 17]             --                   True
├─Bn_bin_conv_pool_block_bw (block1)          [64, 1, 3600]        [64, 8, 898]         --                   True
│    └─ConstantPad1d (pad)                    [64, 1, 3600]        [64, 1, 3610]        --                   --
│    └─BinaryConv1d_bw (conv)                 [64, 1, 3610]        [64, 8, 1802]        56                   True
│    └─MaxPool1d (pool)                       [64, 8, 1802]        [64, 8, 898]         --                   --
│    └─PReLU (prelu)                          [64, 8, 898]         [64, 8, 898]         1                    True
│    └─BatchNorm1d (bn)                       [64, 8, 898]         [64, 8, 898]         16                   True
├─Bn_bin_conv_pool_block_baw (block2)         [64, 8, 898]         [64, 16, 448]       

In [6]:
model_HW.block1.conv = model_best.block1.conv
model_HW.block1.prelu = model_best.block1.prelu
model_HW.block1.bn = model_best.block1.bn

model_HW.block2.conv = model_best.block2.conv
model_HW.block2.prelu = model_best.block2.prelu
model_HW.block2.bn = model_best.block2.bn

model_HW.block3.conv = model_best.block3.conv
model_HW.block3.prelu = model_best.block3.prelu
model_HW.block3.bn = model_best.block3.bn

model_HW.block4.conv = model_best.block4.conv
model_HW.block4.prelu = model_best.block4.prelu
model_HW.block4.bn = model_best.block4.bn

model_HW.block5.conv = model_best.block5.conv
model_HW.block5.prelu = model_best.block5.prelu
model_HW.block5.bn = model_best.block5.bn

model_HW.block6.conv = model_best.block6.conv
model_HW.block6.prelu = model_best.block6.prelu
model_HW.block6.bn = model_best.block6.bn

In [7]:
model_HW.eval()
test_loss, test_acc = 0, 0
correct, total = 0, 0
# Turn on inference context manager
with torch.inference_mode():
    # Loop through DataLoader batches
    for (X, y) in test_loader:
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        test_pred_logits = model_HW(X)
        # 2. Calculate and accumulate loss
        loss = loss_fn(test_pred_logits, y)
        # loss = sigmoid_focal_loss(predicted.float(), y.float(), alpha=0.25, gamma=3, reduction='mean')
        test_loss += loss.item()

        # Calculate and accumulate accuracy
        # test_pred_labels = test_pred_logits.argmax(dim=1)
        # test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))
        _, predicted = torch.max(test_pred_logits.data, dim=1)
        total += len(y)
        correct += (predicted == y).sum().cpu().item()
# Adjust metrics to get average loss and accuracy per batch
test_acc = correct / total
print(test_acc)

0.975


In [8]:
model_best.eval()
test_loss, test_acc = 0, 0
correct, total = 0, 0
# Turn on inference context manager
with torch.inference_mode():
    # Loop through DataLoader batches
    for (X, y) in test_loader:
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        test_pred_logits = model_best(X)
        # 2. Calculate and accumulate loss
        loss = loss_fn(test_pred_logits, y)
        # loss = sigmoid_focal_loss(predicted.float(), y.float(), alpha=0.25, gamma=3, reduction='mean')
        test_loss += loss.item()

        # Calculate and accumulate accuracy
        # test_pred_labels = test_pred_logits.argmax(dim=1)
        # test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))
        _, predicted = torch.max(test_pred_logits.data, dim=1)
        total += len(y)
        correct += (predicted == y).sum().cpu().item()
# Adjust metrics to get average loss and accuracy per batch
test_acc = correct / total
print(test_acc)

0.975


In [9]:
model_best.block1.bn.weight

Parameter containing:
tensor([2.4695, 1.6541, 1.6108, 0.7182, 1.6193, 1.6284, 0.6816, 0.9996],
       device='cuda:0', requires_grad=True)

In [10]:
model_best.block1.bn.running_mean

tensor([2.0936, 2.0657, 0.9983, 1.2805, 1.0693, 1.4606, 0.9864, 1.2323],
       device='cuda:0')

In [11]:
model_best.block1.bn.running_var

tensor([37.4469, 36.5888,  5.0580,  5.2522,  3.4033, 17.3104,  6.3414,  4.8396],
       device='cuda:0')

In [12]:
model_best.block1.bn.bias

Parameter containing:
tensor([ 0.0596, -0.6600, -0.0774, -0.7521,  0.0848, -0.1289, -0.5796, -0.4351],
       device='cuda:0', requires_grad=True)

In [13]:
model_best.block1.bn.running_mean - torch.sqrt(model_best.block1.bn.running_var + model_best.block1.bn.eps)*(model_best.block1.bn.bias/model_best.block1.bn.weight)

tensor([1.9458, 4.4792, 1.1063, 3.6805, 0.9727, 1.7899, 3.1278, 2.1898],
       device='cuda:0', grad_fn=<SubBackward0>)

In [18]:
model = model_best
block = model.block1
block.k = model.block1.bn.weight / torch.sqrt(model_best.block1.bn.running_var + model_best.block1.bn.eps)
block.b = model.block1.bn.bias - model_best.block1.bn.running_mean * model.block1.bn.weight / torch.sqrt(model_best.block1.bn.running_var + model_best.block1.bn.eps)
block.thre_plus = - block.b / block.k
block.thre_minus = block.b / block.k * block.prelu.weight

print(model.block1.k.shape)
print(model.block1.b.shape)
print(block.thre_plus)
print(block.thre_minus)

torch.Size([8])
torch.Size([8])
tensor([1.9458, 4.4792, 1.1063, 3.6805, 0.9727, 1.7899, 3.1278, 2.1898],
       device='cuda:0', grad_fn=<DivBackward0>)
tensor([-1.0501, -2.4172, -0.5970, -1.9862, -0.5249, -0.9659, -1.6879, -1.1817],
       device='cuda:0', grad_fn=<MulBackward0>)


In [2]:
from torchinfo import summary
model_best = torch.load(f'./models/ECG_Net_for_{classes_num}_91.28%.pth')

summary(model=model_best,
        input_size=(batch_size, 1, 3600),  # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])
weightOperation = WeightOperation(model_best)
weightOperation.WeightBinarize()

In [3]:
model_best.block1.conv.weight

Parameter containing:
tensor([[[-1., -1.,  1.,  1.,  1., -1., -1.]],

        [[-1.,  1.,  1.,  1., -1., -1., -1.]],

        [[-1., -1., -1., -1.,  1.,  1.,  1.]],

        [[-1., -1., -1., -1., -1., -1., -1.]],

        [[ 1.,  1.,  1.,  1.,  1., -1., -1.]],

        [[-1.,  1.,  1.,  1.,  1.,  1.,  1.]],

        [[ 1.,  1.,  1.,  1.,  1.,  1.,  1.]],

        [[ 1., -1., -1., -1.,  1., -1., -1.]]], device='cuda:0',
       requires_grad=True)

In [18]:
a = model_best.block1.conv.weight.data

In [19]:
a[a==1]  = 1
a[a==-1] = 0

In [20]:
a=a.reshape((-1,56)).cpu()
a = a.squeeze()
zero_tensor = torch.zeros(8)
a1 = torch.cat((a, zero_tensor))
a1 = a1.reshape((-1,32))
print(a1)

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [21]:
import re
def array_to_txt(arr, filename):
    with open(filename, 'w') as f:
        # 写入每个值
        for i in range(arr.shape[0]):
            row = ''.join(str(int(x)) for x in arr[i])
            # 在每8位后面插入"_"
            row = re.sub("(.{8})", "\\1_", row)
            f.write(row + '\n')

array_to_txt(a1, "block1_weight.txt")